In [106]:
import os
from dotenv import load_dotenv
import re

import dspy

load_dotenv()

api_key = os.environ["GOOGLE_API_KEY"]

lm = dspy.LM(
    model="gemini/gemini-2.0-flash", api_key=api_key, cache=False, temperature=1.0
)
dspy.configure(lm=lm)

In [107]:
class MadLibsTemplateGenerator(dspy.Signature):
    topic: str = dspy.InputField(desc="The topic for the MadLibs template")
    template: str = dspy.OutputField(
        desc="A MadLibs template with placeholders for word types. The placeholders should be within curly brackets and all lower case."
    )
    word_types: list[str] = dspy.OutputField(
        desc="A list of the exact placeholder word types used in the template, in the order they were used"
    )

In [108]:
class MadLibsGenerator(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_template = dspy.Predict(MadLibsTemplateGenerator)

    def forward(self, topic):
        result = self.generate_template(topic=topic)
        return result

In [109]:
class MadLibsApp:
    def __init__(self):
        self.madlibs_generator = MadLibsGenerator()

    def generate_madlib(self, topic: str):
        result = self.madlibs_generator(topic)
        template, placeholder_words = result.template, result.word_types
        print(template)
        pattern = r"\{([^}]+)\}"
        word_types_needed = re.findall(pattern, template)
        if placeholder_words != word_types_needed:
            raise Exception("Mismatch between words extracted and words needed")
        user_inputs = self.collect_user_inputs(placeholder_words=placeholder_words)
        completed_madlib = self.fill_template(
            template=template,
            placeholder_words=placeholder_words,
            user_inputs=user_inputs,
        )
        return completed_madlib

    def collect_user_inputs(self, placeholder_words):
        user_inputs = []
        for word in placeholder_words:
            user_input = input(f"Enter a/an {word}: ")
            user_inputs.append(user_input)
        return user_inputs

    def fill_template(self, template, placeholder_words, user_inputs):
        completed = template

        for word, user_input in zip(placeholder_words, user_inputs):
            placeholder = f"{{{word}}}"
            completed = completed.replace(placeholder, user_input, 1)
        return completed

In [110]:
app = MadLibsApp()
app.generate_madlib(topic="American Civics")

The {adjective} United States of America is a {adjective} country located in {place}. It has a {adjective} system of government with three branches: the {adjective} branch, the {adjective} branch, and the {adjective} branch. Citizens have the right to {verb} and express their {noun}. It's important to be a {adjective} and informed {noun} to participate in our {adjective} democracy.


"The stupid United States of America is a wealthy country located in bumfuck nowhere. It has a oligarchic system of government with three branches: the rich branch, the old branch, and the boomer branch. Citizens have the right to suck and express their love. It's important to be a dumb and informed ignorant to participate in our corrupt democracy."